In [1]:
import urllib.request
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [3]:
raw_data.take(1)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']

In [4]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(',')
    # keep just numeric and logical values
    symbolic_indexes = [1, 2, 3, 41]
    clean_line_split = [item for i, item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

In [5]:
vector_data.take(5)

[array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
        1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00]),
 array([0.00e+00, 2.39e+02, 4.86e+02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.90e+01, 1.90e+01,
        1.00e+00, 0.00e+00, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00]),
 array([0.000e+00, 2.350e+02, 1.337e+03, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00

In [8]:
from pyspark.mllib.stat import Statistics
import math

# column summary statistics 
col_summary = Statistics.colStats(vector_data)

In [11]:
col_summary.take(1)

AttributeError: 'MultivariateStatisticalSummary' object has no attribute 'take'

In [12]:
col_summary.mean()

array([4.79793025e+01, 3.02561030e+03, 8.68532425e+02, 4.45325199e-05,
       6.43292492e-03, 1.41694381e-05, 3.45187755e-02, 1.51815409e-04,
       1.48246734e-01, 1.02121165e-02, 1.11331300e-04, 3.64356981e-05,
       1.13517442e-02, 1.08294992e-03, 1.09307094e-04, 1.00805431e-03,
       0.00000000e+00, 0.00000000e+00, 1.38658073e-03, 3.32285690e+02,
       2.92906557e+02, 1.76686659e-01, 1.76608808e-01, 5.74334087e-02,
       5.77189431e-02, 7.91547343e-01, 2.09823874e-02, 2.89968038e-02,
       2.32470778e+02, 1.88665670e+02, 7.53779698e-01, 3.09057307e-02,
       6.01934756e-01, 6.68350131e-03, 1.76753964e-01, 1.76442621e-01,
       5.81176104e-02, 5.74116687e-02])

In [14]:
col_summary.max()

array([5.8329000e+04, 6.9337564e+08, 5.1554680e+06, 1.0000000e+00,
       3.0000000e+00, 3.0000000e+00, 3.0000000e+01, 5.0000000e+00,
       1.0000000e+00, 8.8400000e+02, 1.0000000e+00, 2.0000000e+00,
       9.9300000e+02, 2.8000000e+01, 2.0000000e+00, 8.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 5.1100000e+02,
       5.1100000e+02, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       2.5500000e+02, 2.5500000e+02, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00])

In [17]:
print("Duration Statistics:")
print("Mean: {}".format(col_summary.mean()[0], 3))
print("St. deviation: {}".format(round(math.sqrt(col_summary.variance()[0]),3)))
print("Max value: {}".format(round(col_summary.max()[0],3)))
print("Min value: {}".format(round(col_summary.min()[0],3)))
print("Total value count: {}".format(col_summary.count()))
print("Number of non-zero values: {}".format(col_summary.numNonzeros()[0]))

Duration Statistics:
Mean: 47.97930249928637
St. deviation: 707.746
Max value: 58329.0
Min value: 0.0
Total value count: 494021
Number of non-zero values: 12350.0


In [18]:
def parse_interaction_with_key(line):
    line_split = line.split(',')
    # keep just numeric and logical values
    symbolic_indexes = [1, 2, 3, 41]
    clean_line_split = [item for i, item in enumerate(line_split) if i not in symbolic_indexes]
    return line_split[41], np.array([float(x) for x in clean_line_split])

label_vector_data = raw_data.map(parse_interaction_with_key)

In [20]:
label_vector_data.take(5)

[('normal.', array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
         1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00])),
 ('normal.', array([0.00e+00, 2.39e+02, 4.86e+02, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.90e+01, 1.90e+01,
         1.00e+00, 0.00e+00, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00])),
 ('normal.',
  array([0.000e+00, 2.350e+02, 1.337e+03, 0.000e+00, 0.000e+00, 0.000e+00

In [24]:
normal_label_data = label_vector_data.filter(lambda x: x[0] == 'normal.')

In [25]:
normal_label_data.take(1)

[('normal.', array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
         1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00]))]

In [27]:
normal_summary = Statistics.colStats(normal_label_data.values())

In [31]:
normal_label_data.take(1)

[('normal.', array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
         1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00]))]

In [32]:
print("Duration Statistics for label: {}".format("normal")) 
print(" Mean: {}".format(normal_summary.mean()[0],3)) 
print(" St. deviation: {}".format(round(math.sqrt(normal_summary.variance()[0]),3))) 
print(" Max value: {}".format(round(normal_summary.max()[0],3))) 
print(" Min value: {}".format(round(normal_summary.min()[0],3))) 
print(" Total value count: {}".format(normal_summary.count())) 
print(" Number of non-zero values: {}".format(normal_summary.numNonzeros()[0])) 

Duration Statistics for label: normal
 Mean: 216.65732231336938
 St. deviation: 1359.213
 Max value: 58329.0
 Min value: 0.0
 Total value count: 97278
 Number of non-zero values: 11690.0


In [34]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key)
    return Statistics.colStats(label_vector_data.values())

normal_sum = summary_by_label(raw_data, 'normal.')
print("Duration Statistics for label: {}".format("normal")) 
print(" Mean: {}".format(normal_summary.mean()[0],3)) 
print(" St. deviation: {}".format(round(math.sqrt(normal_summary.variance()[0]),3))) 
print(" Max value: {}".format(round(normal_summary.max()[0],3))) 
print(" Min value: {}".format(round(normal_summary.min()[0],3))) 
print(" Total value count: {}".format(normal_summary.count())) 
print(" Number of non-zero values: {}".format(normal_summary.numNonzeros()[0])) 


Duration Statistics for label: normal
 Mean: 216.65732231336938
 St. deviation: 1359.213
 Max value: 58329.0
 Min value: 0.0
 Total value count: 97278
 Number of non-zero values: 11690.0


In [35]:
label_list = ["back.","buffer_overflow.","ftp_write.","guess_passwd.",
              "imap.","ipsweep.","land.","loadmodule.","multihop.",
              "neptune.","nmap.","normal.","perl.","phf.","pod.","portsweep.",
              "rootkit.","satan.","smurf.","spy.","teardrop.","warezclient.",
              "warezmaster."]

In [36]:
stats_by_label = [(label, summary_by_label(raw_data, label)) for label in label_list]

In [47]:
duration_by_label = {stat[0]: np.array([float(stat[1].mean()[0]), 
                                         float(math.sqrt(stat[1].variance()[0])), 
                                         float(stat[1].min()[0]), 
                                         float(stat[1].max()[0]), 
                                         int(stat[1].count())]) 
    for stat in stats_by_label}

In [53]:
import pandas as pd
pd.set_option('display.max_columns', 20)

stats_by_label_df = pd.DataFrame.from_dict(duration_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

In [54]:
stats_by_label_df

,Mean,Std Dev,Min,Max,Count
back.,47.979302,707.746472,0.0,58329.0,494021.0
buffer_overflow.,47.979302,707.746472,0.0,58329.0,494021.0
ftp_write.,47.979302,707.746472,0.0,58329.0,494021.0
guess_passwd.,47.979302,707.746472,0.0,58329.0,494021.0
imap.,47.979302,707.746472,0.0,58329.0,494021.0
ipsweep.,47.979302,707.746472,0.0,58329.0,494021.0
land.,47.979302,707.746472,0.0,58329.0,494021.0
loadmodule.,47.979302,707.746472,0.0,58329.0,494021.0
multihop.,47.979302,707.746472,0.0,58329.0,494021.0
neptune.,47.979302,707.746472,0.0,58329.0,494021.0


In [62]:
def get_variable_stats_df(stats_by_label, column_i):
    column_stats_by_label = {stat[0]: np.array([float(stat[1].mean()[column_i]), 
                                         float(math.sqrt(stat[1].variance()[column_i])), 
                                         float(stat[1].min()[column_i]), 
                                         float(stat[1].max()[column_i]), 
                                         int(stat[1].count())]) 
    for stat in stats_by_label}
    return pd.DataFrame.from_dict(column_stats_by_label, 
                                               columns=["Mean", "Std Dev", "Min", "Max", "Count"], 
                                               orient='index')

In [65]:
get_variable_stats_df(stats_by_label, 0)

,Mean,Std Dev,Min,Max,Count
back.,47.979302,707.746472,0.0,58329.0,494021.0
buffer_overflow.,47.979302,707.746472,0.0,58329.0,494021.0
ftp_write.,47.979302,707.746472,0.0,58329.0,494021.0
guess_passwd.,47.979302,707.746472,0.0,58329.0,494021.0
imap.,47.979302,707.746472,0.0,58329.0,494021.0
ipsweep.,47.979302,707.746472,0.0,58329.0,494021.0
land.,47.979302,707.746472,0.0,58329.0,494021.0
loadmodule.,47.979302,707.746472,0.0,58329.0,494021.0
multihop.,47.979302,707.746472,0.0,58329.0,494021.0
neptune.,47.979302,707.746472,0.0,58329.0,494021.0


In [66]:
correlation_matrix = Statistics.corr(vector_data, method="spearman")


In [67]:
col_names = ["duration","src_bytes","dst_bytes","land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

corr_df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
duration,1.000000,0.014196,0.299189,-0.001068,-0.008025,0.017883,0.108639,0.014363,0.159564,0.010687,...,-0.161107,-0.217167,-0.211979,0.231644,-0.065202,0.100692,-0.056753,-0.057298,-0.007759,-0.013891
src_bytes,0.014196,1.000000,-0.167931,-0.009404,-0.019358,0.000094,0.113920,-0.008396,-0.089702,0.118562,...,0.130377,0.741979,0.729151,-0.712965,0.815039,-0.140231,-0.645920,-0.641792,-0.297338,-0.300581
dst_bytes,0.299189,-0.167931,1.000000,-0.003040,-0.022659,0.007234,0.193156,0.021952,0.882185,0.169772,...,-0.611972,0.024124,0.055033,-0.035073,-0.396195,0.578557,-0.167047,-0.158378,-0.003042,0.001621
land,-0.001068,-0.009404,-0.003040,1.000000,-0.000333,-0.000065,-0.000539,-0.000076,-0.002785,-0.000447,...,-0.019923,-0.012341,0.002576,-0.001803,0.004265,0.016171,0.013566,0.012265,0.000389,-0.001816
wrong_fragment,-0.008025,-0.019358,-0.022659,-0.000333,1.000000,-0.000150,-0.004042,-0.000568,-0.020911,-0.003370,...,-0.029149,-0.058225,-0.049560,0.055542,-0.015449,0.007306,0.010387,-0.024117,0.046656,-0.013666
urgent,0.017883,0.000094,0.007234,-0.000065,-0.000150,1.000000,0.008594,0.063009,0.006821,0.031765,...,-0.005894,-0.005698,-0.004078,0.005208,-0.001939,-0.000976,-0.001381,-0.001370,-0.000786,-0.000782
hot,0.108639,0.113920,0.193156,-0.000539,-0.004042,0.008594,1.000000,0.112560,0.189126,0.811529,...,-0.074178,-0.017960,0.018783,-0.017198,-0.086998,-0.014141,-0.004706,-0.010721,0.199019,0.189142
num_failed_logins,0.014363,-0.008396,0.021952,-0.000076,-0.000568,0.063009,0.112560,1.000000,-0.002190,0.004619,...,-0.028369,-0.015092,0.003004,-0.002960,-0.006617,-0.002588,0.014713,0.014914,0.032395,0.032151
logged_in,0.159564,-0.089702,0.882185,-0.002785,-0.020911,0.006821,0.189126,-0.002190,1.000000,0.161190,...,-0.682721,0.080352,0.114526,-0.093565,-0.359506,0.659078,-0.143283,-0.132474,0.007236,0.012979
num_compromised,0.010687,0.118562,0.169772,-0.000447,-0.003370,0.031765,0.811529,0.004619,0.161190,1.000000,...,-0.041615,0.003465,0.038980,-0.039091,-0.078843,-0.020979,-0.005019,-0.004504,0.214115,0.217858


In [68]:
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)

In [74]:
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

,src_bytes,dst_bytes,hot,logged_in,num_compromised,num_outbound_cmds,is_hot_login,count,srv_count,serror_rate,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
src_bytes,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
dst_bytes,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hot,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
logged_in,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
num_compromised,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
num_outbound_cmds,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
is_hot_login,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
count,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
srv_count,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
serror_rate,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [73]:
correlated_vars_index==True

duration                       False
src_bytes                       True
dst_bytes                       True
land                           False
wrong_fragment                 False
urgent                         False
hot                             True
num_failed_logins              False
logged_in                       True
num_compromised                 True
root_shell                     False
su_attempted                   False
num_root                       False
num_file_creations             False
num_shells                     False
num_access_files               False
num_outbound_cmds               True
is_hot_login                    True
is_guest_login                 False
count                           True
srv_count                       True
serror_rate                     True
srv_serror_rate                 True
rerror_rate                     True
srv_rerror_rate                 True
same_srv_rate                   True
diff_srv_rate                   True
s